In [1]:
import process_text
import seq2seq
import numpy as np
from sklearn.model_selection import train_test_split
from importlib import reload

Using TensorFlow backend.
/home/ubuntu/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/.local/lib/python3.6/site-packages/tensorflow/python

/home/ubuntu/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/.local/lib/python3.6/site-packages/tensorboard/compat/ten

In [41]:
reload(process_text)

<module 'process_text' from '/home/ubuntu/How-to-build-own-text-summarizer-using-deep-learning/process_text.py'>

In [2]:
config = process_text.fit_text("phone_reviews_old.csv")

/home/ubuntu/.local/lib/python3.6/site-packages/bs4/__init__.py:375: UserWarning: "https://www.amazon.com/gp/product/b006qnkqou/ref=cm_cr_ryp_prd_ttl_sol_3" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/ubuntu/.local/lib/python3.6/site-packages/bs4/__init__.py:375: UserWarning: "http://www.twitter.com" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/ubuntu/.local/lib/python3.6/site-packages/bs4/__init__.py:375: UserWarning: "https://www.amazon.com/gp/product/b00g197q4m/ref=cm_cr_ryp_prd_ttl_sol_26" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client

/home/ubuntu/.local/lib/python3.6/site-packages/bs4/__init__.py:375: UserWarning: "https://www.amazon.com/dp/b00yd54ayu/ref=cm_cr_ryp_prd_ttl_sol_0" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/ubuntu/.local/lib/python3.6/site-packages/bs4/__init__.py:375: UserWarning: "https://www.amazon.com/dp/b00zvnqaa0/ref=cm_cr_ryp_prd_ttl_sol_5" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/ubuntu/.local/lib/python3.6/site-packages/bs4/__init__.py:375: UserWarning: "https://www.amazon.com/dp/b015ngq9ge/ref=cm_cr_ryp_prd_ttl_sol_3" looks like a URL. Beautiful Soup is not an HTTP client. You should pr

In [7]:
reload(seq2seq)

<module 'seq2seq' from '/home/ubuntu/How-to-build-own-text-summarizer-using-deep-learning/seq2seq.py'>

In [10]:
summarizer = seq2seq.Seq2SeqSummarizer(config)

In [11]:
df = config['reviews_dataframe']

In [12]:
x_tr,x_val,y_tr,y_val=train_test_split(np.array(df['text']),np.array(df['summary']),test_size=0.1,random_state=0,shuffle=True) 

In [ ]:
summarizer.fit(x_tr, y_tr, x_val, y_val, epochs=1, model_dir_path='./models')

Train on 150377 samples, validate on 16709 samples
148352/150377 [============================>.] - ETA: 30s - loss: 1.7484

#Import the Libraries

In [63]:
import numpy as np
import pandas as pd 
import re
import os
from bs4 import BeautifulSoup
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from attention import AttentionLayer
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")

In [64]:
# read dataset
data=pd.read_csv("phone_reviews_old.csv", nrows=50000)

# Drop Duplicates and NA values

In [34]:
data.drop_duplicates(subset=['Text'],inplace=True)#dropping duplicates
data.dropna(inplace=True)#dropping na

In [35]:
# remove rows where the summary has the word start and the length is 2 words
data = data[data['Summary'].apply(lambda x: not(len(x.lower().split(' ')) <= 2 and ('star' in x.lower().split(' ') or 'stars' in x.lower().split(' ')) ))]

In [36]:
data = data[~data.Text.str.match('<.*>')]

# Information about dataset

Let us look at datatypes and shape of the dataset

In [37]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have"}

In [38]:
stop_words = set(stopwords.words('english')) 

def text_cleaner(text,num):
    newString = text.lower()
    newString = BeautifulSoup(newString, "html.parser").text ##ME - had to change from lxml to html.parser
    newString = re.sub(r'\([^)]*\)', '', newString)
    newString = re.sub('"','', newString)
    newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])    
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z0-9]", " ", newString) 
    newString = re.sub('[m]{2,}', 'mm', newString)
    if(num==0):
        tokens = [w for w in newString.split() if not w in stop_words]
    else:
        tokens=newString.split()
    long_words=[]
    for i in tokens:
        if len(i)>1:                                                 #removing short word
            long_words.append(i)   
    return (" ".join(long_words)).strip()

In [39]:
#call the function
cleaned_text = []
for t in data['Text']:
    cleaned_text.append(text_cleaner(t,0)) 

Let us look at the first five preprocessed reviews

In [40]:
#call the function
cleaned_summary = []
for t in data['Summary']:
    cleaned_summary.append(text_cleaner(t,1))

Let us look at the first 10 preprocessed summaries

In [41]:
data['cleaned_text']=cleaned_text
data['cleaned_summary']=cleaned_summary

#Drop empty rows

In [42]:
data.replace('', np.nan, inplace=True)
data.dropna(axis=0,inplace=True)

In [43]:
# remove the rows where the review text has less than or equal to 3 words
data = data[data['cleaned_text'].apply(lambda x: len(x.split(' ')) > 3)]

In [44]:
max_text_len=100
max_summary_len=20

Let us select the reviews and summaries whose length falls below or equal to **max_text_len** and **max_summary_len**

In [45]:
cleaned_text =np.array(data['cleaned_text'])
cleaned_summary=np.array(data['cleaned_summary'])

short_text=[]
short_summary=[]

for i in range(len(cleaned_text)):
    if(len(cleaned_summary[i].split())<=max_summary_len and len(cleaned_text[i].split())<=max_text_len):
        short_text.append(cleaned_text[i])
        short_summary.append(cleaned_summary[i])
        
df=pd.DataFrame({'text':short_text,'summary':short_summary})

In [46]:
df['summary'] = df['summary'].apply(lambda x : 'sostok '+ x + ' eostok')

We are getting closer to the model building part. Before that, we need to split our dataset into a training and validation set. We’ll use 90% of the dataset as the training data and evaluate the performance on the remaining 10% (holdout set):

In [47]:
from sklearn.model_selection import train_test_split
x_tr,x_val,y_tr,y_val=train_test_split(np.array(df['text']),np.array(df['summary']),test_size=0.1,random_state=0,shuffle=True) 

In [48]:
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences

#prepare a tokenizer for reviews on training data
x_tokenizer = Tokenizer() 
x_tokenizer.fit_on_texts(list(x_tr))

In [49]:
thresh=4

cnt=0
tot_cnt=0
freq=0
tot_freq=0

for key,value in x_tokenizer.word_counts.items():
    tot_cnt=tot_cnt+1
    tot_freq=tot_freq+value
    if(value<thresh):
        cnt=cnt+1
        freq=freq+value
    
print("% of rare words in vocabulary:",(cnt/tot_cnt)*100)
print("Total Coverage of rare words:",(freq/tot_freq)*100)

% of rare words in vocabulary: 66.65586519766688
Total Coverage of rare words: 3.43591101566239


In [50]:
#prepare a tokenizer for reviews on training data
x_tokenizer = Tokenizer(num_words=tot_cnt-cnt) 
x_tokenizer.fit_on_texts(list(x_tr))

#convert text sequences into integer sequences
x_tr_seq    =   x_tokenizer.texts_to_sequences(x_tr) 
x_val_seq   =   x_tokenizer.texts_to_sequences(x_val)

#padding zero upto maximum length
x_tr    =   pad_sequences(x_tr_seq,  maxlen=max_text_len, padding='post')
x_val   =   pad_sequences(x_val_seq, maxlen=max_text_len, padding='post')

#size of vocabulary ( +1 for padding token)
x_tokenizer_word_index = x_tokenizer.word_index

#Summary Tokenizer

In [51]:
#prepare a tokenizer for reviews on training data
y_tokenizer = Tokenizer()   
y_tokenizer.fit_on_texts(list(y_tr))

In [52]:
thresh=6

cnt=0
tot_cnt=0
freq=0
tot_freq=0

for key,value in y_tokenizer.word_counts.items():
    tot_cnt=tot_cnt+1
    tot_freq=tot_freq+value
    if(value<thresh):
        cnt=cnt+1
        freq=freq+value
    
print("% of rare words in vocabulary:",(cnt/tot_cnt)*100)
print("Total Coverage of rare words:",(freq/tot_freq)*100)

% of rare words in vocabulary: 76.45687645687646
Total Coverage of rare words: 4.644989607191559


Let us define the tokenizer with top most common words for summary.

In [53]:
#prepare a tokenizer for reviews on training data
y_tokenizer = Tokenizer(num_words=tot_cnt-cnt) 
y_tokenizer.fit_on_texts(list(y_tr))

#convert text sequences into integer sequences
y_tr_seq    =   y_tokenizer.texts_to_sequences(y_tr) 
y_val_seq   =   y_tokenizer.texts_to_sequences(y_val) 

#padding zero upto maximum length
y_tr    =   pad_sequences(y_tr_seq, maxlen=max_summary_len, padding='post')
y_val   =   pad_sequences(y_val_seq, maxlen=max_summary_len, padding='post')

#size of vocabulary
y_tokenizer_word_index = y_tokenizer.word_index

In [54]:
ind=[]
for i in range(len(y_tr)):
    cnt=0
    for j in y_tr[i]:
        if j!=0:
            cnt=cnt+1
    if(cnt==2):
        ind.append(i)

y_tr=np.delete(y_tr,ind, axis=0)
x_tr=np.delete(x_tr,ind, axis=0)

In [55]:
ind=[]
for i in range(len(y_val)):
    cnt=0
    for j in y_val[i]:
        if j!=0:
            cnt=cnt+1
    if(cnt==2):
        ind.append(i)

y_val=np.delete(y_val,ind, axis=0)
x_val=np.delete(x_val,ind, axis=0)

### Glove embeddings

In [56]:
embeddings_index = {}
f = open(os.path.join('/dev/summarization', 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()


In [57]:
embedding_dim=100
embedding_matrix_x = np.zeros((len(x_tokenizer_word_index)+1, embedding_dim))
for word, i in x_tokenizer_word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix_x[i] = embedding_vector

In [58]:
embedding_matrix_y = np.zeros((len(y_tokenizer_word_index)+1, embedding_dim))
for word, i in y_tokenizer_word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix_y[i] = embedding_vector

# Model building

We are finally at the model building part. But before we do that, we need to familiarize ourselves with a few terms which are required prior to building the model.

**Return Sequences = True**: When the return sequences parameter is set to True, LSTM produces the hidden state and cell state for every timestep

**Return State = True**: When return state = True, LSTM produces the hidden state and cell state of the last timestep only

**Initial State**: This is used to initialize the internal states of the LSTM for the first timestep

**Stacked LSTM**: Stacked LSTM has multiple layers of LSTM stacked on top of each other. 
This leads to a better representation of the sequence. I encourage you to experiment with the multiple layers of the LSTM stacked on top of each other (it’s a great way to learn this)

Here, we are building a 3 stacked LSTM for the encoder:

## Bidirectional

In [59]:
from keras import backend as K 
K.clear_session()
import tensorflow as tf
from tensorflow.core.protobuf import rewriter_config_pb2
from tensorflow.keras.backend import set_session
tf.keras.backend.clear_session()  # For easy reset of notebook state.

config_proto = tf.ConfigProto()
off = rewriter_config_pb2.RewriterConfig.OFF
config_proto.graph_options.rewrite_options.arithmetic_optimization = off
session = tf.Session(config=config_proto)
set_session(session)

latent_dim = 300
embedding_dim=100

# Encoder
encoder_inputs = Input(shape=(max_text_len,))

#embedding layer
enc_emb = Embedding(len(x_tokenizer_word_index)+1,
                    embedding_dim,
                    weights=[embedding_matrix_x],
                    input_length=max_text_len,
                    trainable=False)(encoder_inputs)
#enc_emb =  Embedding(x_voc, embedding_dim,trainable=True,name='encoder_emb_layer')(encoder_inputs)

#encoder lstm 1
encoder_lstm1 = Bidirectional(LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4),name='encoder_lstm_layer1')
encoder_output1, forward_h1, forward_c1, backward_h1, backward_c1 = encoder_lstm1(enc_emb)

#encoder lstm 2
encoder_lstm2 = Bidirectional(LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4),name='encoder_lstm_layer2')
encoder_output2, forward_h2, forward_c2, backward_h2, backward_c2 = encoder_lstm2(encoder_output1)

#encoder lstm 3
encoder_lstm3=Bidirectional(LSTM(latent_dim, return_state=True, return_sequences=True,dropout=0.4,recurrent_dropout=0.4), name='encoder_lstm_layer3')
encoder_outputs, forward_h, forward_c, backward_h, backward_c= encoder_lstm3(encoder_output2)


state_h = Concatenate(axis=-1)([forward_h, backward_h])
state_c = Concatenate(axis=-1)([forward_c, backward_c])
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))

#embedding layer
dec_emb_layer = Embedding(len(y_tokenizer_word_index)+1,
                    embedding_dim,
                    weights=[embedding_matrix_y],
                    input_length=max_summary_len,
                    trainable=False)
#dec_emb_layer = Embedding(y_voc, embedding_dim,trainable=True,name='decoder_emb_layer')
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = LSTM(latent_dim*2, return_sequences=True, return_state=True,dropout=0.4,recurrent_dropout=0.2,name='decoder_lstm_layer')
decoder_outputs,decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb,initial_state=encoder_states)

# Attention layer
attn_layer = AttentionLayer(name='attention_layer')
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])

# Concat attention input and decoder LSTM output
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

#dense layer
decoder_dense =  TimeDistributed(Dense(len(y_tokenizer_word_index)+1, activation='softmax'))
decoder_outputs = decoder_dense(decoder_concat_input)

# Define the model 
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.summary() 

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 100)     1543100     input_1[0][0]                    
__________________________________________________________________________________________________
encoder_lstm_layer1 (Bidirectio [(None, 100, 600), ( 962400      embedding[0][0]                  
__________________________________________________________________________________________________
encoder_lstm_layer2 (Bidirectio [(None, 100, 600), ( 2162400     encoder_lstm_layer1[0][0]        
______________________________________________________________________________________________

In [60]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

In [61]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=2)

In [62]:
history=model.fit([x_tr,y_tr[:,:-1]], y_tr.reshape(y_tr.shape[0],y_tr.shape[1], 1)[:,1:] ,epochs=3,callbacks=[es],batch_size=128, validation_data=([x_val,y_val[:,:-1]], y_val.reshape(y_val.shape[0],y_val.shape[1], 1)[:,1:]))

Train on 18471 samples, validate on 2045 samples
Epoch 1/3
18471/18471 [==============================] - 404s 22ms/sample - loss: 1.9447 - val_loss: 1.6115
Epoch 2/3
18471/18471 [==============================] - 394s 21ms/sample - loss: 1.6062 - val_loss: 1.4947
Epoch 3/3
18471/18471 [==============================] - 394s 21ms/sample - loss: 1.5139 - val_loss: 1.4345


In [24]:
model.save('phone_reviews_bidirectional_epoch3.h5')

NameError: name 'model' is not defined

In [26]:
from tensorflow.keras.models import load_model
newmodel = load_model('./phone_reviews_bidirectional_epoch3.h5', custom_objects={'AttentionLayer': AttentionLayer})

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [25]:
reverse_target_word_index=y_tokenizer.index_word
reverse_source_word_index=x_tokenizer.index_word
target_word_index=y_tokenizer.word_index

In [27]:
encoder_inputs = newmodel.input
decoder_inputs = newmodel.get_layer('input_2').input
encoder_outputs, forward_h, forward_c, backward_h, backward_c = newmodel.get_layer('encoder_lstm_layer3').output

In [32]:
dec_emb_layer = newmodel.get_layer('decoder_emb_layer').output

# Inference

Set up the inference for the encoder and decoder:

## Bidirectional inference

In [34]:
from keras import backend as K 
K.clear_session()

latent_dim = 300
embedding_dim=100
# Encode the input sequence to get the feature vector
encoder_model = Model(inputs=encoder_inputs,outputs=[encoder_outputs, forward_h, forward_c, backward_h, backward_c])

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim*2,))
decoder_state_input_c = Input(shape=(latent_dim*2,))
decoder_hidden_state_input = Input(shape=(max_text_len,latent_dim*2))

# Get the embeddings of the decoder sequence
dec_emb_layer = Embedding(len(y_tokenizer_word_index)+1,
                            embedding_dim,
                            weights=[embedding_matrix_y],
                            input_length=max_summary_len,
                            trainable=False,
                            name='decoder_emb_layer')
dec_emb2= dec_emb_layer(decoder_inputs) 
# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

#attention inference

attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])
decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs2, attn_out_inf])

# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_inf_concat) 

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2] + [state_h2, state_c2])

ValueError: Tensor("Cast:0", shape=(?, ?), dtype=int32) must be from the same graph as Tensor("decoder_emb_layer/embeddings/Read/ReadVariableOp:0", shape=(1310, 100), dtype=float32).

## Bidirectional decoder

In [31]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    e_out, forward_h, forward_c, backward_h, backward_c = encoder_model.predict(input_seq)
    
    e_h = np.concatenate((forward_h, backward_h), axis=-1)
    e_c = np.concatenate((forward_c, backward_c), axis=-1)
    

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    
    # Populate the first word of target sequence with the start word.
    target_seq[0, 0] = target_word_index['sostok']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
      
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index[sampled_token_index]
        
        if(sampled_token!='eostok'):
            decoded_sentence += ' '+sampled_token

        # Exit condition: either hit max length or find stop word.
        if (sampled_token == 'eostok'  or len(decoded_sentence.split()) >= (max_summary_len-1)):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        e_h, e_c = h, c

    return decoded_sentence

In [32]:
def seq2summary(input_seq):
    newString=''
    for i in input_seq:
        if((i!=0 and i!=target_word_index['sostok']) and i!=target_word_index['eostok']):
            newString=newString+reverse_target_word_index[i]+' '
    return newString

def seq2text(input_seq):
    newString=''
    for i in input_seq:
        if(i!=0):
            newString=newString+reverse_source_word_index[i]+' '
    return newString

In [95]:
e_out, forward_h, forward_c, backward_h, backward_c = encoder_model.predict(x_tr[i].reshape(1,max_text_len))
    
e_h = np.concatenate((forward_h, backward_h), axis=-1)
e_c = np.concatenate((forward_c, backward_c), axis=-1)

In [33]:
for i in range(0,100):
    print("Review:",seq2text(x_tr[i])) 
    print("Original summary:",seq2summary(y_tr[i]))
    print("Predicted summary:",decode_sequence(x_tr[i].reshape(1,max_text_len)))
    print("\n")

Review: best phone ever would recommend anyone second thoughts buy phone without hesitation disappointed samsung galaxy note rise beyond expectations wow say 
Original summary: best phone ever 
Predicted summary:  good phone


Review: used awesome got crappy seven months got replaced exact issues supper buggy phone get get mad 
Original summary: it used to be awesome but it got crappy after months 
Predicted summary:  good phone


Review: liked weight shape screen like speaker sometimes hard understand voice talking 
Original summary: liked the weight 
Predicted summary:  the phone is not good but it was


Review: manual came unit incomplete still unable operate phone 
Original summary: useless purchase 
Predicted summary:  good phone


Review: thank loving phone kinda hole online thing glad 
Original summary: kinda doing the online thing but am glad did 
Predicted summary:  great phone


Review: first phone received work place purchased prompt replacing got second phone happy place pu

Predicted summary:  the phone is not good but it is


Review: want run phone best model choose flashing touch took minutes running great 
Original summary: is here 
Predicted summary:  good phone


Review: bought friend verizon send new sim cost registering verizon easy enough nice inexpensive upgrade entry level moto android phone bigger screen storage faster cpu lte great value stuck verizon 
Original summary: great value 
Predicted summary:  it is good phone but it is


Review: turning know worst experience 
Original summary: its worst experience 
Predicted summary:  the phone was not good but it was


Review: great deal looks works like brand new got son christmas loves never even knew brand new came charger earphones like order new phone could happier 
Original summary: even better than expected 
Predicted summary:  great phone


Review: excellent budget phone care service provider 
Original summary: am glad bought this phone 
Predicted summary:  great phone


Review: horrible pho

Predicted summary:  great phone


Review: love phone works great amazon said used add line cannot tell difference 
Original summary: love this phone 
Predicted summary:  great phone


Review: slot damaged working proceeded update firmware lenovo smart assistant continues damaged means use use one sim calls charger also arrived damaged disappointed lose live venezuela unfortunately return phone 
Original summary: damaged 
Predicted summary:  the phone is not good but it was


Review: phone supposed brand new could tell case works fine brand new 
Original summary: it works fine but was not brand new 
Predicted summary:  good phone


Review: great phone know latest thing bought cause budget great screen beautiful display buyer excellent well 
Original summary: great phone know its not the latest thing 
Predicted summary:  great phone


Review: excellent quality good memory capacity many basic apps already included fair price 
Original summary: excellent iphone 4s good quality 
Predicted s

In [72]:
for i in range(0,100):
    print("Review:",seq2text(x_tr[i])) 
    print("Original summary:",seq2summary(y_tr[i]))
    print("Predicted summary:",decode_sequence(x_tr[i].reshape(1,max_text_len)))
    print("\n")

Review: best phone ever would recommend anyone second thoughts buy phone without hesitation disappointed samsung galaxy note rise beyond expectations wow say 
Original summary: best phone ever 
Predicted summary:  great phone


Review: used awesome got crappy seven months got replaced exact issues supper buggy phone get get mad 
Original summary: it used to be awesome but it got crappy after months 
Predicted summary:  great phone


Review: liked weight shape screen like speaker sometimes hard understand voice talking 
Original summary: liked the weight 
Predicted summary:  the phone


Review: manual came unit incomplete still unable operate phone 
Original summary: useless purchase 
Predicted summary:  great phone


Review: thank loving phone kinda hole online thing glad 
Original summary: kinda doing the online thing but am glad did 
Predicted summary:  great phone


Review: first phone received work place purchased prompt replacing got second phone happy place purchased phone suppos

Predicted summary:  great phone


Review: bought friend verizon send new sim cost registering verizon easy enough nice inexpensive upgrade entry level moto android phone bigger screen storage faster cpu lte great value stuck verizon 
Original summary: great value 
Predicted summary:  great phone


Review: turning know worst experience 
Original summary: its worst experience 
Predicted summary:  it


Review: great deal looks works like brand new got son christmas loves never even knew brand new came charger earphones like order new phone could happier 
Original summary: even better than expected 
Predicted summary:  great phone


Review: excellent budget phone care service provider 
Original summary: am glad bought this phone 
Predicted summary:  great phone


Review: horrible phone buy end call button middle screen phone face contact hangs phone talk speaker phone 
Original summary: horrible phone do not buy it 
Predicted summary:  the phone


Review: purchased phone teenage daughter b

Predicted summary:  the phone


Review: phone supposed brand new could tell case works fine brand new 
Original summary: it works fine but was not brand new 
Predicted summary:  great phone


Review: great phone know latest thing bought cause budget great screen beautiful display buyer excellent well 
Original summary: great phone know its not the latest thing 
Predicted summary:  great phone


Review: excellent quality good memory capacity many basic apps already included fair price 
Original summary: excellent iphone 4s good quality 
Predicted summary:  great phone


Review: loved received worked perfectly days phone started overheat almost hot put ear made phone calls battery would run within hours contacted seller breed twice seller respond mails may buy another nokia phone liked dealing seller avoid breed 
Original summary: the battery would last only hours and the seller did not even bother to respond to my complaint 
Predicted summary:  great phone


Review: old school guy decen

### Applying to video reviews

In [34]:
# get the sentences wit positive or negative sentiment from text
from sentiment_filter import SentimentFilter

In [43]:
text = "(mellow piano music) (upbeat music) - This is the GoPro MAX. And it's GoPro's second attempt at a consumer level, 360 camera. (camera whirring) Last year, Insta360 came out with the One X, and it totally revolutionized the idea of what a 360 camera could be used for. First off, footage shot with the One X was stitched in camera, and then you were able to edit it right on your phone and upload to social platforms almost instantly. Secondly, they market it not as a 360-degree camera but instead a camera that could shoot 360 degrees of possible angles. Now fast forward a year and we have the GoPro Max, which is the successor to the GoPro Fusion that came out in 2017, that was filled with all of the headaches of 360's past. It required two memory cards. You had to stitch the footage on a desktop. But they overhauled all of that with the Max, and they in turn made a really accessible 360 camera, which makes me believe I might continue to actually use this thing. ♪ We wag, we wag, we wag, we wag, we wag ♪ ♪ This is the widest angle GoPro's made ♪ ♪ This is the wide ♪ The GoPro MAX has dual 180-degree lenses that shoot 16.6-megapixel, 360-degree photos, 5.5-megapixel single lens, or GoPro is calling Max SuperView photos, and the super cool 6.2-megapixel panoramic photos called PowerPanos. Video-wise, you have 1440p60 video with a single lens and 5.6K30 spherical video, just like the Insta360. There are mics on all but one side and it shows the same super satisfying rubber buttons and button layout as the Hero8. On the bottom, you have these new stowable mounting prongs, which alas, you no longer need that housing, which I somehow always manage to forget anyways, and it also makes it compatible with all GoPro mounts. The touchscreen on the Max is a bit smaller than the touchscreen on the back of the Hero, but its menu system is identical. The big plus here is if you're vlogging or taking a selfie, you can see yourself much like on the DJI Osmo Action because, well, there's lenses on both sides. The Max is waterproof up to 16 feet. But that proofing is merely for protection at this point. GoPro claims that it's really difficult to stitch underwater footage, but they are promising some sort of waterproof housing in the future. The screen is super responsive, but you can't scroll around your frame in 360 mode since the screen has swiping gestures such as swiping down to reach the main menu. You can however switch between lenses but only while not recording. Once you hit that record button, the screen has no functionality other than just being a viewfinder to the lens that you chose before rolling. The image on the screen will however stay level with the horizon no matter which way you turn the camera. And it does so with almost no noticeable lag. If you lean into that warped, super fisheye look that only a 360 camera could give you, the footage is sick. Skin tones are true to life and it's not afraid to overexpose the highlights of it. I prefer this to the flatter color grade of the One X, especially for users who won't be color correcting. And how even if you are color correcting, I find that the Max and the One X have very little latitude in the coloring process. Now, if you're only gonna be showing these photos on social platforms that you're gonna look at on a phone, who cares, but for a camera that costs $500, I hope it's going a little further than that. (laughs). The Max's 360° video is stabilized in software. It bobs up and down as you'd expect from walking and it can be really jittery, especially at night. The stitching is most noticeable at the top and the bottom of the frame. And since most of GoPro's mounts typically have a wider base than the camera itself, there's almost always a bit of artifact from the mount in the footage. Over-all though, once you export the 360-degree footage, the stitch is pretty clean. The audio, however, is probably the best I've seen on a camera this size. The wind reduction can add a lot of compression, and enough wind can sound like, well this, (wind blowing) Guys, when I say it's windy out, it's like actually very windy out. I am about to blow over. This might make some really good footage as I fly away. But when you're not in 25-mile an hour wind, it actually reduces the sound of wind while prioritizing voices. So while hiking with Alex, I love that the GoPro picked up on the crickets and footsteps, while also keeping our voices at the forefront no matter where the camera was in space. ♪ Yeah, and I think it's gonna be a long, long time ♪ And then there are those PowerPano photos. I gotta be honest guys, I had a tough time not showing you these photos before this video dropped. I mean first and foremost, they make taking a panoramic photo a lot easier. And you don't have to stand there and slowly and robotically move your phone across the landscape. I was really surprised at how crisp these photos turned out to. Even on a desktop, they still look really cool. The time warp feature in 360 mode, which is when the Max just records the time lapse, is really cool, but it's a total battery killer. I saw my battery drop 15% off a fresh charge while shooting this time warp. And I let it record for all of 10 minutes. I think it would help if the screen would go off a bit sooner while shooting these to save some battery. Otherwise, the battery performance is on par with the Hero8. Even though it is a different battery. So yes, you will need to buy extra spares if you plan on heavy use. I was able to get through a full day of heavy use using two of these batteries. 360 cameras are only accessible though if you can do something with the footage. So the software has to be sound, and GoPro's app makes it really easy to edit this footage. Max uses the same GoPro app as the Heroes. Now, I'm no stranger to keyframes, but I think even a casual non-video director from the verge.com could edit the Max's 360 footage. And I actually enjoyed editing it on the app. GoPro also has a 360 desktop app called GoPro Player, which mirrors the mobile app experience with added codec options at export. You have H.264, ProRes, HUVC, and as far as 360 editing goes, I was blown away by its ease. Okay so there's this whole other mode on the Max called Hero mode. And it's basically just when the camera's using one lens as opposed to two. It has more mic controls and it even boasts GoPro's most stable video called Max HyperSmooth. And it also has the widest field of view called Max SuperView. But I gotta be honest, while it does sound good and the footage is stable, the fact that it maxes out at 1440p60 is noticeable. Especially in how much smoothing is being done when the subject is close to the camera. I also noticed the camera struggled to keep the footage level with the horizon, choosing to instead focus on keeping the footage stable. So when climbing down this trail, which to be fair, is a slanted surface, the horizon kept tilting further and further from level to almost a dizzying point. GoPro has become a household name. I mean they're specced out, durable, tiny, portable beasts of cameras. And the Max is no different. And while the Max has great features such as PowerPano, it also has great hardware and a really good touch screen. But for me, it came down to the ease of use of the app that made me wanna keep using this thing. 360 footage used to mean high-end equipment and tons of headaches and post, and that always pushed me away from using it. But when you put all of that into the form factor of a GoPro, a workflow I've been using for years, it's way more approachable. Now, is all of that enough to make this thing more than a novelty camera that will ultimately end up in my forgotten type bed in my basement? Time will truly tell. But it is the most fun and user-friendly 360 camera I have used today. And I'm excited to see this thing as specced out. (coughing) 4K Hero mode? Please? All right, so for 499, you can pre-order a Max today. They start shipping on October 24th and will be on shelves October 27th. The 360 space is getting kinda nuts"

In [29]:
text = " Google has been building its own Chromebooks for a while now first there were two iterations of the Chromebook pixel and then there was 2017 pixel book all three worked great laptops with one huge flaw they cost way too much money Google's back at it again this year with the new pixel book go and for the first time Google's not trying to make the best Chromebook ever price be damned [Music] instead Google is trying to build a Chromebook that anyone can use and afford the pixel will go starts at six hundred and fifty dollars a full 350 less than the original pixel book it's still a lot of money for a Chromebook but for the first time Google's built something better than the cheap low-end Chromebooks out there that still semi affordable unfortunately in its quest to get the price down Google also second rest a lot of what made the original pixel looks so intriguing in the first place but for a lot of people those trade-offs will be worth it all of Google's previous Chromebooks as well as last year's ill-fated pixel slate tablet had distinctive designs and premium hardware with very few corners cut while the pixel book though is a nice device it's a phone or pedestrian computer than anything Google has released thus far it's a basic looking clamshell laptop with a 13.3 inch 1080p touchscreen and a black paint job unmarked except for a small Google Gee logo in the corner of the lid it also comes in very light pink with a bright coral underside a much more distinctive combo but the all-black has a certain understated elegance either way the go feels sturdy and well built but doesn't have any of the distinctive design elements you'll see in the original pixel book it's thicker than that device still weighs about the same at 2.3 pounds I definitely miss the aesthetics of the original though particularly it's glass an aluminum lid that echoed the design of goulds pixel phones the go as a functional utilitarian design which I can appreciate but I wish it was slightly more exciting there is one new funky design flourish to be found the pixel book go it's ridged underside for whatever reason pictures made me think that this was a different softer material than the magnesium that makes up the rest of the goes case but it doesn't feel any different than the rest of the device google says it design needs word just to make the laptop easier to grip or pull out of a sleeve but I can't have had any problems handling other computers still it's good to see Google's more playful side on this otherwise dull laptop and while the goal might be a little boring it still feels nicer and sturdier than almost any other Chromebook on the market the otherwise excellent Asus Chromebook flip that I reviewed earlier this year it's much heavier than the go for example Google's surely simplified its design as a way to cut costs and as such there are some other notable changes to the go compared to its predecessor unlike the first pixel book it's not a convertible so tablet mode is out that also means it doesn't work with the pixel book pen stylus both the pixel book in the pixel slate at excellent displays with a taller aspect ratio and higher resolutions in the pixel book go and I'm a little sad to see the step back and display quality here that's not to say the Dell has a bad screen it's just not nearly as gorgeous to look at as those last two computers 1080p on a 13.3 inch screen works out to 166 pixels per inch a far cry from the 235 PPI on the first pixel book and the impressive 293 PPI on the pixel sleep Google did build a go with a 4k screen but that option is only available if you get a top-of-the-line model which costs an eye-popping thirteen hundred and ninety nine dollars the good news is that the go as much slimmer bezels around the screen than the original pixel book but I still prefer the taller and more pixel dense display on that laptop to the screen on the go fortunately Google didn't skimp out on the keyboard the original pixel book offered a great typing experience and the go is almost as good I can't quite put my finger on what's different but the keys don't feel quite as soft when you're typing that set there's plenty of travel and they're pretty quiet Google about the whole thing right here the trackpad under their hand is just fine it feels kind of rough under my finger and scrolling around isn't as smooth as I'd like it's not bad and I've gotten used to it but I've definitely used better touch beds well google cut some corners in the design department the pixel book go is still among the more powerful Chromebooks out there the base model pairs an eighth generation Intel Core m3 processor with 8 gigabytes of RAM and 64 gigabytes of storage the one I reviewed steps up to an i5 processor in 128 gigabytes of storage that model costs $849 comparable to what defined in the nine $99 pixel look I wish I could have tried that m3 model because I really want to see what kind of performance Google is offering for $650 unsurprisingly Intel's Core i5 and eight gigs of ram is plenty for Chrome OS the pixel book go handle my normal workload without any issues my routine involves a dozen or so browser tabs TweetDeck hangouts slack todoist google keep and Trello all running as Chrome apps along with the occasional Android app or to Adobe's Lightroom Netflix and YouTube music all mostly worked well although music playback had the occasional low memory hiccup which is frustrating notably this didn't happen when using YouTube music as a browser tab this reinforces my belief that Android apps on Chrome aren't usually worth using if there's a web-based alternative overall I can't imagine most people need to upgrade from the $849 model I tried Google was also selling in 999 configuration that doubles the range of 16 gigabytes well that $13.99 model I mentioned includes a core i7 processor 16 gigs of ram 206 gigs of storage and a 4k display well I'd surely enjoy that high res screen I don't think it's a worthwhile upgrade for anyone the pixel book though also holds up pretty well in the battery side something that wasn't always true with the original and our video playback test the go ran for just over 13 and a half hours before shutting down that's more than an hour longer than the 12 hour battery life google claims with mixed usage unfortunately I never came close to 12 hours and putting the go through my normal routine but it did last more than eight hours which is a definite improvement over the first pixel book all this said the pixel book go is a much easier device to evaluate than either the first pixel book or the pixel Slaten it's a standard familiar almost boring laptop but instead of figuring out if cromoz works on a convertible tablet or if you're willing to spend $1000 on a premium laptop the go is just a straightforward no-frills device it's still not particularly cheap especially if you step up to the i5 processor I tried but premium Chromebooks in the 600 to 800 dollar range are becoming more common now and now Google has something that can compete on quality as well as price yes you can still get a Chromebook for less money than what Google was asked for the NGO if you think a Chromebook can be your main computer and don't mind paying a little extra for a device that's thin light and well engineered the pixel book go will likely meet your needs thanks for watching and subscribe to and gadgets YouTube channel for more news and reviews"

In [35]:
text = "- Alright, it's iPhone time. Apple released three new iPhones all at once this year. The iPhone 11, the iPhone 11 Pro, and the iPhone 11 Pro Max. Now, I honestly think the iPhone 11 is the phone most people in the iOS ecosystem should get if they're upgrading. So we've got a whole other review of that phone that you should check out, too. But this, this is the Pro review. The dark mode, cash money review for professionals who use their phones to do pro stuff. Like taking telephoto photos. Actually, I have no idea why Apple called these phones Pro. They're basically just nice updates to the iPhone XS, but I also think it's a waste of time to argue about names. iPhone 11 Pro Max is a bad name. But you know what, it's a great phone. And I think Apple might have done it. This is the best camera I've ever seen on a phone. Let's get into it. (calming music) The iPhone 11 Pro looks almost exactly like the iPhone XS from the front. It's a little heavier and thicker, but unless you're comparing them directly, you probably won't notice. I certainly didn't. You do get a much bigger battery in exchange for that extra size, which Apple says leads to a four-hour battery life increase on the regular Pro and a five-hour jump for the Pro Max. But it's surprisingly hard to check that number. I'll get into why in a minute. The big difference from the XS comes on the back where Apple says the rear glass is now stronger and it comes in this frosted matte finish. It also integrates a glossy camera bump with three cameras. The whole back and the bump is a single piece of glass that's milled out. The iPhone 11 is the reverse. It has a glossy back and the camera bump is matte. Now you know. I got a lot of questions about this matte finish and I think it's really nice, especially in this new midnight green color. It didn't really pick up any fingerprints, but it does seem a tiny bit more slippery than the gloss back of the previous phones and the iPhone 11. But I also think most people are just going to stick this thing in a case, so it doesn't seem like a huge issue. A lot of people also asked me if this matte finish will scratch easily, similar to the back of the Pixel 3, and so far the answer seems to be no. But we're going to have to keep an eye on it over time. Apple's finally relented and included an 18W USB-C fast charger with the Pro phones. So you get a USB-C to Lightning cable as well. Now Apple's USB-C charger is not the smallest or prettiest charger out there. But anything is better than the slow 5W brick it's been including for years, so I'll take it. So let's get into these cameras. Apple has a lot at stake here. Google and Samsung's cameras have been outperforming the iPhone for a couple of years now. And Chinese phone makers, in particular, have been racing each other to add an endless array of photography features. If you're in the US, you're probably locked into iMessage and no camera is good enough to make you switch. But Apple doesn't have that advantage in big markets like Europe and China. Everyone uses WhatsApp and WeChat, and switching from iOS to Android is much easier. So Apple's added a bunch of features that first appeared on Android phones. There are three cameras on the iPhone 11 Pro. The telephoto camera has the same basic sensor, but a faster f/2.O lens. The main camera has the same f/1.8 lens and a slightly better sensor. And there's the new super fun ultra-wide camera, which is basically twice as wide as the main lens and f/2.4. And the f/2.2 front camera is now 12 megapixels, up from seven, and has a wider 24mm focal length so it can pull out to a slightly wider angle in landscape for selfies. Apple's also updated its smart HDR processing system, which I was not very impressed with when I reviewed the iPhone XS and XR. The iPhone 11 cameras are an enormous improvement over the XS, and beat the Pixel and Galaxy Note 10 in most of our tests. This shot of Verge video producer, Mariya Abdulkaf, outside in bright sunlight shouldn't be too hard for any camera to do a good job with. And all these photos look basically fine. But zoom and do 100 percent crop and the improvement from the iPhone XS to the iPhone 11 main camera is stunning. The iPhone 11 is way sharper, with way more detail. I also think the iPhone 11 does a way better job than the Note 10 and it's a little better to my eye than the Pixel 3. This improvement is due to something Apple's calling semantic rendering. Basically, Smart HDR recognizes what's in the image and renders it appropriately. I asked Apple to break down how it works for me and it basically goes like this: First, the iPhone starts taking photos to a buffer the instant you open the camera app. So by the time you actually press the button, it's captured four underexposed frames and the photo you want. Then it grabs one overexposed frame. This is all basically the same as the iPhone XS and the Pixel 3, except the Pixel doesn't grab that overexposed frame. Second, Smart HDR and semantic rendering then start looking for things in the photos it understands. Faces, hair, the sky, things like that. Then it uses additional detail from the under and overexposed frames to selectively process the areas of the image it's recognized. Hair gets sharpened, but the sky doesn't. It just gets de-noised. Faces get relighted to make them look more even and the iPhone knows to sharpen up your facial hair. Smart HDR is also less aggressive in flattening the photos. Highlights and faces aren't corrected as aggressively as before because those highlights make photos look more natural. But other highlights and shadows are corrected to regain detail. Finally, the whole image is saved and you've got a photo. This all happens instantly every time you take a photo, which is a testament to how powerful Apple's A13 processor really is. Here's that final image we just took. You can see how the iPhone 11 preserves more detail than the Pixel and Note in the shadows and highlights, and just blows the iPhone XS away in sharpness. The improvements to Smart HDR are applicable across cameras. Here's a quick comparison of the ultra-wides in the 11 Pro and the Note 10. The Note 10 did a better job exposing the sky, but the 11 is just a much sharper photo with more detail. You can see the same thing with selfies. The XS Max has pulled up shadows so much that it looks a little hazy, while the iPhone 11 evenly pulls up shadows in Mariya's face, but preserves the contrast with the background. The Note 10 did all kinds of goofy skin smoothing and the Pixel 3 looks great, but it has less detail than the iPhone. Just look at her eyelashes. Now there are places where the iPhone 11 didn't win in our tests. The XS historically struggled with backlit subjects. And while the 11 is a definite improvement, this photo is really blue. And there's basically no detail in Mariya's face. I think the Note actually did the best overall job here. And I think the Pixel does the best job in low light, but the iPhone isn't bad. It's a lot better than the XS was last year. Apple's promised a new mode for these situations called Deep Fusion, which is supposed to ship later this year, but I wasn't able to test it. If it improves photos taken in this kind of light, then the iPhone will basically pull ahead in every situation. Apple also added a Night mode this year and it's really impressive. It preserves a ton more detail than the Pixel. It comes on automatically in the dark and suggests an exposure time, which you can tweak or turn off if you want. The Pixel photo here looks more dramatic, but look a little closer. The iPhone has preserved all of the detail in the brick and graffiti, which is basically gone in the Pixel shot. Both of these Night mode photos look pretty similar, but look at all the extra detail in the bushes in the iPhone shot. Same with this shot. You'd never know that fence was even there in the Pixel photo. I think the Pixel produces more dramatic-looking shots out of the camera, but I would rather have the extra detail from the iPhone. All in all, I think the iPhone 11 camera does better in bright light than the Pixel 3 and Note 10, and the Night mode beats the Pixel 3 more often than not. If this promised Deep Fusion update improves medium- to low-light performance as much as Apple says it will, the iPhone 11 will once again be the smartphone camera to beat. Apple's also made video on the iPhone 11 better with improved stabilization exposure and the ability to record 4K60 video from any of the three cameras. - But is it really that good? Let's put it to the test. Alright, so we're going to head out in New York City and we're going to test against the Note 10, the Pixel 3, and the XS to see who does video best. We started out at the water and immediately you can see the color differences between the cameras. The Pixel leaned into the warm tones, while Samsung muddied the mid tones. And the iPhone XS leaned into the blues of the sky, while the 11 Pro balanced the warm and cool tones out the most. With the subject closer to the lens the Note 10 kept the flowers the sharpest, while the iPhone XS blurred the background the most. Okay, we've got to test the audio on all these phones. I'll let you guys be the judge. Do I sound like the beautiful angel voice that I have, angelic voice? Of course, the answer is yes. But on these phones, who's to say? In low light, Apple isn't afraid of underexposing the blacks, whereas Samsung and Google will bring them up. Now the 11 Pro does do a lot more smoothing to compensate for all of the noise this typically creates. The front-facing camera on the iPhone 11 Pro did a great job of balancing my skin tone and just the exposure in general. Even against this large neon light source. But it wasn't as sharp as the Note 10. Now all these comparisons are great, but chances are you won't be recording on all of these phones side by side at the same exact time. So let's just talk about the iPhone 11 Pro. The front-facing camera now records in 4K and the slofie is exactly how you'd imagine it. You'll use it once and probably never touch it again. The video from the rear cameras, though, is really good. Like, in perfect light, it's honestly just impressive. Switching between lenses is pretty seamless in terms of color accuracy and exposure. And you can use the dial method or just tap the lenses on the screen to switch. Now you can't switch between lenses when you shoot in 4K60. So make sure you set 4K30 or lower if you're going to be doing that. The ultra-wide angle lens looks super cool even with the crazy distortion. But there is no image stabilization, which shouldn't be a problem because naturally, the wider the lens, the less need for that. Both the telephoto and wide lens keep your shot stable and pretty sharp for a sensor their size. The color is punchy and pretty saturated and in perfect lighting, you, too, can have that shot on iPhone look. But in low light, even the iPhone 11 Pro, when viewed on a screen larger than its own, succumbs to noisy blacks and haloed highlights. My big issue, though, why do I have to leave the camera app to change the frame rates? It's not intuitive and it's just time-consuming. Apple, just put it in the app. iPhones have always had great video, but the video on the 11 Pro is the best I've seen on a smartphone to date. And the gap is only getting bigger. Enough of me, back to Nilay. - The screen on the iPhone 11 Pro is a new OLED that Apple is calling the Super Retina XDR display. There's still a notch and Apple is still doing fancy, rounded corners and uniform bezels, which no one else in the industry has really managed to match. Those bezels are still pretty big, though, especially compared to something like the Galaxy Note 10. This is the third year of the basic iPhone X design and Apple's competitors haven't been sitting still in trying to beat it. The big upgrades to the screen are around brightness and power efficiency. Basically, the screen can get a lot brighter than before and it also uses up to 15 percent less power, according to Apple. Now in typical situations, Apple says the iPhone 11 Pro display can go up to 800 nits of brightness and that when you're watching HDR content, the highlights will peak at 1,200 nits. In normal situations, I don't think you're going to notice it. I've never had a problem looking at my iPhone XS display outside. But it's a huge and obvious improvement when you're watching HDR movies. I never really bought the idea that previous iPhone displays were Dolby Vision, but the iPhone 11 Pro display kind of pulls it off. It's hard to capture this on video, but it's just a lot brighter and punchier on the 11 Pro display compared to the XS. The Note 10 display definitely looks as bright, but the iPhone's color processing looks way more natural to me, especially with Apple's True Tone system turned on. Of course, Apple and Google are still in a fight over video codecs, so you can't watch any 4K or HDR content on the iPhone 11 Pro from YouTube. So this video is not in 4K if you're watching on an iPhone. Sorry about that. Apple has also updated the audio on the iPhone 11 and iPhone 11 Pro, with something they're calling spatial audio. It's basically a surround decoder. If you're watching a Netflix movie with Adobe 5.1 sound or game encoded in 7.1, you'll get virtual surround from the stereo speakers. The iPhone 11s also support Dolby Atmos, which is a little silly for stereo speakers, but it's there. There's also standard wide stereo support like last year for everything else. All in all, the iPhone speakers sound really loud and generally better than ever. There's no more 3D Touch on the iPhone 11 Pro, it's been replaced by what Apple calls Haptic Touch, which is basically just long pressing on things with haptic feedback. In most places, you won't really notice it's gone and some things are actually a little easier to figure out. Opening the camera from the lock screen feels the same to me. Rearranging icons on the home screen is a little simpler to figure out since there's a context menu that pops up now. Peeking at links in Safari is a little different in iOS 13, but it's also a little simpler. The only place I truly missed 3D Touch was the keyboard. You could press down anywhere on the keyboard to move the cursor around on the XS, but with Haptic Touch you can only press and hold on the space bar. Now, I never really realized how many times a day I used that one specific 3D Touch feature and I missed it. But if the trade-off is that there's more room inside the case for a bigger battery, I'm fine with it. Apple claims the iPhone 11 Pro lasts four hours longer than the iPhone XS, and that the iPhone 11 Pro Max lasts five hours longer than the XS Max. As I mentioned earlier, it's really hard to test those claims. Apple doesn't run a strict battery test that we can just repeat. Instead, it's getting that number by taking its huge data set of iPhone usage and applying it to a model of the new phone. So you'll notice that the only hard numbers Apple publishes are like hours of video playback. Simple tests of a single task, which isn't how anyone uses a phone. All that said, the battery life on the iPhone 11 Pro has been impressive. I've been using an 11 Pro Max as my primary phone for the past week, and it has consistently run for 12 to 14 hours on a single charge. And about 10 hours of screen-on time, off the charger, is reported in battery settings. That is a huge improvement over my XS Max, which generally runs for eight to nine hours The smaller 11 Pro has been just as impressive. That's the phone Becca used for her testing and it still had 50 percent of its battery left the morning after she shot her part of the review. It turns out a slightly thicker phone with a bigger battery was the right move after all. Inside the new iPhone 11 Pro, there's the new A13 Bionic processor which, as usual, benchmarks far above the competition. There's a lot of headroom here for things like AR, games, and high-end photo and video apps, but it's not like the A12 Bionic in the iPhone XS is slow. Most of this extra power will be useful towards the end of this phone's life, not the beginning. Face ID is a little bit faster and works at more angles, but it's not particularly dramatic, it just works a little bit better. I think this is the best kind of iterative update. It made a thing that was already pretty good a little bit better, and anything that makes securing your phone a little bit better is great. And most intriguingly, there's a new chip inside the iPhone 11, it's called the U1, which does precise positioning using an ultra wideband radio. Apple says the forthcoming iOS 13.1 update will allow you to just point the phone at another U1 device to put it at the top of your AirDrop list. But none of that works yet, so we'll just have to see. Obviously, the iPhone 11 Pro runs iOS 13 and I have to tell you, iOS 13 is pretty buggy. I saw all kinds of glitches and crashes during my week of testing, and iOS 13.1 is already in beta and scheduled for wide release by the end of this month. So it really seems like Apple just squeaked this thing out the door and is racing to fix bugs. I asked Apple about some of these glitches and they told me they're always fixing bugs and iPhone owners generally auto update. But if you're on the fence about upgrading, you might want to wait until some of these bug fix updates actually ship. Assuming all the bugs get fixed, the biggest new feature in iOS 13 is Dark Mode, which is very nice. But otherwise, this is a lot of tiny little updates. There's a swiping keyboard now. Reminders is a much better app. There are better photo editing tools and you can edit videos as well. Apple Maps keeps getting a tiny bit better every year. Siri's voice is slightly smoother. It's a lot of little things that add up into a big update, but if you handed this phone to someone using iOS 12, they might not even notice unless Dark Mode was on. The iPhone 11 Pro starts at $999 with 64GB of storage, which doesn't seem like quite enough for that price. And the 11 Pro Max starts at $1,099. You can spec the Pro Max all the way up to $1,449 with 512GB of storage, which means that you can pay an awful lot of money for this phone if you want to. If you're in the iOS ecosystem and it's time to upgrade, it's a pretty tough choice this year. The standard iPhone 11 offers almost everything you get from the Pro for $699. And I think most people should get the iPhone 11. You'll still get the improved main camera, the fun new ultra-wide lens, the A13 processor, and iOS 13, and all the rest. The extra money for the Pro basically buys you a far superior display, a telephoto camera, and improved LTE performance. And if you want a smaller phone, the iPhone 11 Pro is a little bit smaller than the iPhone 11, which seems a tiny bit unfair to people who want a smaller phone. The entire lineup is still pretty big, though. So if you're hoping for something to replace that aging iPhone SE, you're not going to love any of these options. I'm very picky about displays and cameras, so I'm going to get an iPhone 11 Pro. In fact, the cameras on the iPhone 11 and 11 Pro are so improved that I think they're worth the year-over-year upgrade from last year's models for the first time in a long time. Add in the improved battery life and the iPhone 11 Pro stands out as a major step forward from the XS and one of the best flagship phones of the year. Or it will be, when Apple fixes iOS 13. Hey everybody, this was obviously the iPhone 11 Pro review. We also reviewed the iPhone 11, that was a fun collab we did with Dieter Bohn. Watch both videos, they're both on the channel, and then let me know which of these phones you think you're going to buy or if you're waiting for the Pixel 4."

In [36]:
# using regex to remove any character that is not alpha numeric or punctuation
import re
pattern = r"[^{}]".format('a-zA-Z0-9!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~') 
text = re.sub(pattern,' ',text)

In [37]:
test = SentimentFilter(text)
test.getSentiment()
test.filterPositives(0.9)
test.filterNegatives(0.9)

In [38]:
test.getFilteredTokens()

["- Alright, it's iPhone time.",
 'Apple released three new iPhones all at once this year.',
 'The iPhone 11, the iPhone 11 Pro, and the iPhone 11 Pro Max.',
 "Now, I honestly think the iPhone 11 is the phone most people in the iOS ecosystem should get if they're upgrading.",
 "So we've got a whole other review of that phone that you should check out, too.",
 'But this, this is the Pro review.',
 'The dark mode, cash money review for professionals who use their phones to do pro stuff.',
 'Actually, I have no idea why Apple called these phones Pro.',
 "But you know what, it's a great phone.",
 'And I think Apple might have done it.',
 "This is the best camera I've ever seen on a phone.",
 "It's a little heavier and thicker, but unless you're comparing them directly, you probably won't notice.",
 "I'll get into why in a minute.",
 'It also integrates a glossy camera bump with three cameras.',
 'The iPhone 11 is the reverse.',
 'Now you know.',
 "I got a lot of questions about this matte 

In [39]:
# cleaning up review sentences
cleaned_review_text = []
for t in test.getFilteredTokens():
    cleaned_text = text_cleaner(t,0)
    if cleaned_text and len(cleaned_text.split(' ')) > 4:
        cleaned_review_text.append(cleaned_text)
        
# trying to test with the whole text
#cleaned_review_text.append(text_cleaner(text,0))

# convert cleaned text to numpy array
reviews = np.array(cleaned_review_text)

In [40]:
# tokenize test data
x_val_seq   =   x_tokenizer.texts_to_sequences(reviews)
x_val   =   pad_sequences(x_val_seq, maxlen=max_text_len, padding='post')

In [48]:
from tensorflow.keras.models import load_model
newmodel = load_model('./phone_reviews_bilstm_epoch20.h5', custom_objects={'AttentionLayer': AttentionLayer})

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [49]:
newmodel.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 100)     1373600     input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional (Bidirectional)   [(None, 100, 600), ( 962400      embedding[0][0]                  
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) [(None, 100, 600), ( 2162400     bidirectional[0][0]              
______________________________________________________________________________________________

In [50]:
newmodel.layers

In [71]:

encoder_outputs, forward_h, forward_c, backward_h, backward_c = newmodel.get_layer('bidirectional_2').output

In [72]:
encoder_inputs = newmodel.get_layer('input_1').output

In [73]:
decoder_inputs = newmodel.get_layer('input_2').output

In [41]:
for index in range(len(cleaned_review_text)):
    print("Review:",cleaned_review_text[index])
    print("Predicted summary:",decode_sequence(x_val[index].reshape(1,max_text_len)))

Review: apple released three new iphones year
Predicted summary:  the phone was not good but it was
Review: iphone 11 iphone 11 pro iphone 11 pro max
Predicted summary:  great phone
Review: honestly think iphone 11 phone people ios ecosystem get upgrading
Predicted summary:  good phone
Review: got whole review phone check
Predicted summary:  it is good phone but it is
Review: dark mode cash money review professionals use phones pro stuff
Predicted summary:  the phone was not good but it was
Review: actually idea apple called phones pro
Predicted summary:  the phone was not good but it was
Review: best camera ever seen phone
Predicted summary:  good phone
Review: little heavier thicker unless comparing directly probably notice
Predicted summary:  good phone
Review: also integrates glossy camera bump three cameras
Predicted summary:  it is good phone but it is
Review: got lot questions matte finish think really nice especially new midnight green color
Predicted summary:  good phone
Revie

Predicted summary:  good phone
Review: phone becca used testing still 50 percent battery left morning shot part review
Predicted summary:  the phone is good but the phone is not
Review: turns slightly thicker phone bigger battery right move
Predicted summary:  the phone is not good but it is
Review: inside new iphone 11 pro new a13 bionic processor usual benchmarks far competition
Predicted summary:  good phone
Review: face id little bit faster works angles particularly dramatic works little bit better
Predicted summary:  great phone
Review: think best kind iterative update
Predicted summary:  good phone
Review: made thing already pretty good little bit better anything makes securing phone little bit better great
Predicted summary:  good phone
Review: intriguingly new chip inside iphone 11 called u1 precise positioning using ultra wideband radio
Predicted summary:  good phone
Review: apple says forthcoming ios 13 update allow point phone another u1 device put top airdrop list
Predicted

In [89]:
for index in range(len(cleaned_review_text)):
    print("Review:",cleaned_review_text[index])
    print("Predicted summary:",decode_sequence(x_val[index].reshape(1,max_text_len)))

Review: first two iterations chromebook pixel pixel book three worked great laptops one huge flaw cost way much money
Predicted summary:  great laptop
Review: instead google trying build chromebook anyone use afford pixel go starts six hundred fifty dollars full less original pixel book
Predicted summary:  the battery life is not as good as it is
Review: first time google built something better cheap low end chromebooks still semi affordable unfortunately quest get price google
Predicted summary:  the best thing have ever owned
Review: trade offs worth google previous chromebooks well last year ill fated pixel slate tablet distinctive designs premium hardware corners cut pixel book though nice device
Predicted summary:  great chromebook
Review: phone pedestrian computer anything google released thus far
Predicted summary:  five stars
Review: much distinctive combo black certain understated elegance either way go feels sturdy well built distinctive design elements see original pixel boo

#### PreProcessing of the review to get the subject sentences and then run inference

In [20]:
import spacy
import sentences
from importlib import reload

In [24]:
reload(sentences)

<module 'sentences' from '/home/ubuntu/How-to-build-own-text-summarizer-using-deep-learning/sentences.py'>

In [53]:
sent = set()
text = "- Alright, it's iPhone time. Apple released three new iPhones all at once this year. The iPhone 11, the iPhone 11 Pro, and the iPhone 11 Pro Max. Now, I honestly think the iPhone 11 is the phone most people in the iOS ecosystem should get if they're upgrading. So we've got a whole other review of that phone that you should check out, too. But this, this is the Pro review. The dark mode, cash money review for professionals who use their phones to do pro stuff. Like taking telephoto photos. Actually, I have no idea why Apple called these phones Pro. They're basically just nice updates to the iPhone XS, but I also think it's a waste of time to argue about names. iPhone 11 Pro Max is a bad name. But you know what, it's a great phone. And I think Apple might have done it. This is the best camera I've ever seen on a phone. Let's get into it. (calming music) The iPhone 11 Pro looks almost exactly like the iPhone XS from the front. It's a little heavier and thicker, but unless you're comparing them directly, you probably won't notice. I certainly didn't. You do get a much bigger battery in exchange for that extra size, which Apple says leads to a four-hour battery life increase on the regular Pro and a five-hour jump for the Pro Max. But it's surprisingly hard to check that number. I'll get into why in a minute. The big difference from the XS comes on the back where Apple says the rear glass is now stronger and it comes in this frosted matte finish. It also integrates a glossy camera bump with three cameras. The whole back and the bump is a single piece of glass that's milled out. The iPhone 11 is the reverse. It has a glossy back and the camera bump is matte. Now you know. I got a lot of questions about this matte finish and I think it's really nice, especially in this new midnight green color. It didn't really pick up any fingerprints, but it does seem a tiny bit more slippery than the gloss back of the previous phones and the iPhone 11. But I also think most people are just going to stick this thing in a case, so it doesn't seem like a huge issue. A lot of people also asked me if this matte finish will scratch easily, similar to the back of the Pixel 3, and so far the answer seems to be no. But we're going to have to keep an eye on it over time. Apple's finally relented and included an 18W USB-C fast charger with the Pro phones. So you get a USB-C to Lightning cable as well. Now Apple's USB-C charger is not the smallest or prettiest charger out there. But anything is better than the slow 5W brick it's been including for years, so I'll take it. So let's get into these cameras. Apple has a lot at stake here. Google and Samsung's cameras have been outperforming the iPhone for a couple of years now. And Chinese phone makers, in particular, have been racing each other to add an endless array of photography features. If you're in the US, you're probably locked into iMessage and no camera is good enough to make you switch. But Apple doesn't have that advantage in big markets like Europe and China. Everyone uses WhatsApp and WeChat, and switching from iOS to Android is much easier. So Apple's added a bunch of features that first appeared on Android phones. There are three cameras on the iPhone 11 Pro. The telephoto camera has the same basic sensor, but a faster f/2.O lens. The main camera has the same f/1.8 lens and a slightly better sensor. And there's the new super fun ultra-wide camera, which is basically twice as wide as the main lens and f/2.4. And the f/2.2 front camera is now 12 megapixels, up from seven, and has a wider 24mm focal length so it can pull out to a slightly wider angle in landscape for selfies. Apple's also updated its smart HDR processing system, which I was not very impressed with when I reviewed the iPhone XS and XR. The iPhone 11 cameras are an enormous improvement over the XS, and beat the Pixel and Galaxy Note 10 in most of our tests. This shot of Verge video producer, Mariya Abdulkaf, outside in bright sunlight shouldn't be too hard for any camera to do a good job with. And all these photos look basically fine. But zoom and do 100 percent crop and the improvement from the iPhone XS to the iPhone 11 main camera is stunning. The iPhone 11 is way sharper, with way more detail. I also think the iPhone 11 does a way better job than the Note 10 and it's a little better to my eye than the Pixel 3. This improvement is due to something Apple's calling semantic rendering. Basically, Smart HDR recognizes what's in the image and renders it appropriately. I asked Apple to break down how it works for me and it basically goes like this: First, the iPhone starts taking photos to a buffer the instant you open the camera app. So by the time you actually press the button, it's captured four underexposed frames and the photo you want. Then it grabs one overexposed frame. This is all basically the same as the iPhone XS and the Pixel 3, except the Pixel doesn't grab that overexposed frame. Second, Smart HDR and semantic rendering then start looking for things in the photos it understands. Faces, hair, the sky, things like that. Then it uses additional detail from the under and overexposed frames to selectively process the areas of the image it's recognized. Hair gets sharpened, but the sky doesn't. It just gets de-noised. Faces get relighted to make them look more even and the iPhone knows to sharpen up your facial hair. Smart HDR is also less aggressive in flattening the photos. Highlights and faces aren't corrected as aggressively as before because those highlights make photos look more natural. But other highlights and shadows are corrected to regain detail. Finally, the whole image is saved and you've got a photo. This all happens instantly every time you take a photo, which is a testament to how powerful Apple's A13 processor really is. Here's that final image we just took. You can see how the iPhone 11 preserves more detail than the Pixel and Note in the shadows and highlights, and just blows the iPhone XS away in sharpness. The improvements to Smart HDR are applicable across cameras. Here's a quick comparison of the ultra-wides in the 11 Pro and the Note 10. The Note 10 did a better job exposing the sky, but the 11 is just a much sharper photo with more detail. You can see the same thing with selfies. The XS Max has pulled up shadows so much that it looks a little hazy, while the iPhone 11 evenly pulls up shadows in Mariya's face, but preserves the contrast with the background. The Note 10 did all kinds of goofy skin smoothing and the Pixel 3 looks great, but it has less detail than the iPhone. Just look at her eyelashes. Now there are places where the iPhone 11 didn't win in our tests. The XS historically struggled with backlit subjects. And while the 11 is a definite improvement, this photo is really blue. And there's basically no detail in Mariya's face. I think the Note actually did the best overall job here. And I think the Pixel does the best job in low light, but the iPhone isn't bad. It's a lot better than the XS was last year. Apple's promised a new mode for these situations called Deep Fusion, which is supposed to ship later this year, but I wasn't able to test it. If it improves photos taken in this kind of light, then the iPhone will basically pull ahead in every situation. Apple also added a Night mode this year and it's really impressive. It preserves a ton more detail than the Pixel. It comes on automatically in the dark and suggests an exposure time, which you can tweak or turn off if you want. The Pixel photo here looks more dramatic, but look a little closer. The iPhone has preserved all of the detail in the brick and graffiti, which is basically gone in the Pixel shot. Both of these Night mode photos look pretty similar, but look at all the extra detail in the bushes in the iPhone shot. Same with this shot. You'd never know that fence was even there in the Pixel photo. I think the Pixel produces more dramatic-looking shots out of the camera, but I would rather have the extra detail from the iPhone. All in all, I think the iPhone 11 camera does better in bright light than the Pixel 3 and Note 10, and the Night mode beats the Pixel 3 more often than not. If this promised Deep Fusion update improves medium- to low-light performance as much as Apple says it will, the iPhone 11 will once again be the smartphone camera to beat. Apple's also made video on the iPhone 11 better with improved stabilization exposure and the ability to record 4K60 video from any of the three cameras. - But is it really that good? Let's put it to the test. Alright, so we're going to head out in New York City and we're going to test against the Note 10, the Pixel 3, and the XS to see who does video best. We started out at the water and immediately you can see the color differences between the cameras. The Pixel leaned into the warm tones, while Samsung muddied the mid tones. And the iPhone XS leaned into the blues of the sky, while the 11 Pro balanced the warm and cool tones out the most. With the subject closer to the lens the Note 10 kept the flowers the sharpest, while the iPhone XS blurred the background the most. Okay, we've got to test the audio on all these phones. I'll let you guys be the judge. Do I sound like the beautiful angel voice that I have, angelic voice? Of course, the answer is yes. But on these phones, who's to say? In low light, Apple isn't afraid of underexposing the blacks, whereas Samsung and Google will bring them up. Now the 11 Pro does do a lot more smoothing to compensate for all of the noise this typically creates. The front-facing camera on the iPhone 11 Pro did a great job of balancing my skin tone and just the exposure in general. Even against this large neon light source. But it wasn't as sharp as the Note 10. Now all these comparisons are great, but chances are you won't be recording on all of these phones side by side at the same exact time. So let's just talk about the iPhone 11 Pro. The front-facing camera now records in 4K and the slofie is exactly how you'd imagine it. You'll use it once and probably never touch it again. The video from the rear cameras, though, is really good. Like, in perfect light, it's honestly just impressive. Switching between lenses is pretty seamless in terms of color accuracy and exposure. And you can use the dial method or just tap the lenses on the screen to switch. Now you can't switch between lenses when you shoot in 4K60. So make sure you set 4K30 or lower if you're going to be doing that. The ultra-wide angle lens looks super cool even with the crazy distortion. But there is no image stabilization, which shouldn't be a problem because naturally, the wider the lens, the less need for that. Both the telephoto and wide lens keep your shot stable and pretty sharp for a sensor their size. The color is punchy and pretty saturated and in perfect lighting, you, too, can have that shot on iPhone look. But in low light, even the iPhone 11 Pro, when viewed on a screen larger than its own, succumbs to noisy blacks and haloed highlights. My big issue, though, why do I have to leave the camera app to change the frame rates? It's not intuitive and it's just time-consuming. Apple, just put it in the app. iPhones have always had great video, but the video on the 11 Pro is the best I've seen on a smartphone to date. And the gap is only getting bigger. Enough of me, back to Nilay. - The screen on the iPhone 11 Pro is a new OLED that Apple is calling the Super Retina XDR display. There's still a notch and Apple is still doing fancy, rounded corners and uniform bezels, which no one else in the industry has really managed to match. Those bezels are still pretty big, though, especially compared to something like the Galaxy Note 10. This is the third year of the basic iPhone X design and Apple's competitors haven't been sitting still in trying to beat it. The big upgrades to the screen are around brightness and power efficiency. Basically, the screen can get a lot brighter than before and it also uses up to 15 percent less power, according to Apple. Now in typical situations, Apple says the iPhone 11 Pro display can go up to 800 nits of brightness and that when you're watching HDR content, the highlights will peak at 1,200 nits. In normal situations, I don't think you're going to notice it. I've never had a problem looking at my iPhone XS display outside. But it's a huge and obvious improvement when you're watching HDR movies. I never really bought the idea that previous iPhone displays were Dolby Vision, but the iPhone 11 Pro display kind of pulls it off. It's hard to capture this on video, but it's just a lot brighter and punchier on the 11 Pro display compared to the XS. The Note 10 display definitely looks as bright, but the iPhone's color processing looks way more natural to me, especially with Apple's True Tone system turned on. Of course, Apple and Google are still in a fight over video codecs, so you can't watch any 4K or HDR content on the iPhone 11 Pro from YouTube. So this video is not in 4K if you're watching on an iPhone. Sorry about that. Apple has also updated the audio on the iPhone 11 and iPhone 11 Pro, with something they're calling spatial audio. It's basically a surround decoder. If you're watching a Netflix movie with Adobe 5.1 sound or game encoded in 7.1, you'll get virtual surround from the stereo speakers. The iPhone 11s also support Dolby Atmos, which is a little silly for stereo speakers, but it's there. There's also standard wide stereo support like last year for everything else. All in all, the iPhone speakers sound really loud and generally better than ever. There's no more 3D Touch on the iPhone 11 Pro, it's been replaced by what Apple calls Haptic Touch, which is basically just long pressing on things with haptic feedback. In most places, you won't really notice it's gone and some things are actually a little easier to figure out. Opening the camera from the lock screen feels the same to me. Rearranging icons on the home screen is a little simpler to figure out since there's a context menu that pops up now. Peeking at links in Safari is a little different in iOS 13, but it's also a little simpler. The only place I truly missed 3D Touch was the keyboard. You could press down anywhere on the keyboard to move the cursor around on the XS, but with Haptic Touch you can only press and hold on the space bar. Now, I never really realized how many times a day I used that one specific 3D Touch feature and I missed it. But if the trade-off is that there's more room inside the case for a bigger battery, I'm fine with it. Apple claims the iPhone 11 Pro lasts four hours longer than the iPhone XS, and that the iPhone 11 Pro Max lasts five hours longer than the XS Max. As I mentioned earlier, it's really hard to test those claims. Apple doesn't run a strict battery test that we can just repeat. Instead, it's getting that number by taking its huge data set of iPhone usage and applying it to a model of the new phone. So you'll notice that the only hard numbers Apple publishes are like hours of video playback. Simple tests of a single task, which isn't how anyone uses a phone. All that said, the battery life on the iPhone 11 Pro has been impressive. I've been using an 11 Pro Max as my primary phone for the past week, and it has consistently run for 12 to 14 hours on a single charge. And about 10 hours of screen-on time, off the charger, is reported in battery settings. That is a huge improvement over my XS Max, which generally runs for eight to nine hours The smaller 11 Pro has been just as impressive. That's the phone Becca used for her testing and it still had 50 percent of its battery left the morning after she shot her part of the review. It turns out a slightly thicker phone with a bigger battery was the right move after all. Inside the new iPhone 11 Pro, there's the new A13 Bionic processor which, as usual, benchmarks far above the competition. There's a lot of headroom here for things like AR, games, and high-end photo and video apps, but it's not like the A12 Bionic in the iPhone XS is slow. Most of this extra power will be useful towards the end of this phone's life, not the beginning. Face ID is a little bit faster and works at more angles, but it's not particularly dramatic, it just works a little bit better. I think this is the best kind of iterative update. It made a thing that was already pretty good a little bit better, and anything that makes securing your phone a little bit better is great. And most intriguingly, there's a new chip inside the iPhone 11, it's called the U1, which does precise positioning using an ultra wideband radio. Apple says the forthcoming iOS 13.1 update will allow you to just point the phone at another U1 device to put it at the top of your AirDrop list. But none of that works yet, so we'll just have to see. Obviously, the iPhone 11 Pro runs iOS 13 and I have to tell you, iOS 13 is pretty buggy. I saw all kinds of glitches and crashes during my week of testing, and iOS 13.1 is already in beta and scheduled for wide release by the end of this month. So it really seems like Apple just squeaked this thing out the door and is racing to fix bugs. I asked Apple about some of these glitches and they told me they're always fixing bugs and iPhone owners generally auto update. But if you're on the fence about upgrading, you might want to wait until some of these bug fix updates actually ship. Assuming all the bugs get fixed, the biggest new feature in iOS 13 is Dark Mode, which is very nice. But otherwise, this is a lot of tiny little updates. There's a swiping keyboard now. Reminders is a much better app. There are better photo editing tools and you can edit videos as well. Apple Maps keeps getting a tiny bit better every year. Siri's voice is slightly smoother. It's a lot of little things that add up into a big update, but if you handed this phone to someone using iOS 12, they might not even notice unless Dark Mode was on. The iPhone 11 Pro starts at $999 with 64GB of storage, which doesn't seem like quite enough for that price. And the 11 Pro Max starts at $1,099. You can spec the Pro Max all the way up to $1,449 with 512GB of storage, which means that you can pay an awful lot of money for this phone if you want to. If you're in the iOS ecosystem and it's time to upgrade, it's a pretty tough choice this year. The standard iPhone 11 offers almost everything you get from the Pro for $699. And I think most people should get the iPhone 11. You'll still get the improved main camera, the fun new ultra-wide lens, the A13 processor, and iOS 13, and all the rest. The extra money for the Pro basically buys you a far superior display, a telephoto camera, and improved LTE performance. And if you want a smaller phone, the iPhone 11 Pro is a little bit smaller than the iPhone 11, which seems a tiny bit unfair to people who want a smaller phone. The entire lineup is still pretty big, though. So if you're hoping for something to replace that aging iPhone SE, you're not going to love any of these options. I'm very picky about displays and cameras, so I'm going to get an iPhone 11 Pro. In fact, the cameras on the iPhone 11 and 11 Pro are so improved that I think they're worth the year-over-year upgrade from last year's models for the first time in a long time. Add in the improved battery life and the iPhone 11 Pro stands out as a major step forward from the XS and one of the best flagship phones of the year. Or it will be, when Apple fixes iOS 13. Hey everybody, this was obviously the iPhone 11 Pro review. We also reviewed the iPhone 11, that was a fun collab we did with Dieter Bohn. Watch both videos, they're both on the channel, and then let me know which of these phones you think you're going to buy or if you're waiting for the Pixel 4."
doc = nlp(text)
for token in doc:
    #print(token)
    if token.text.lower() in ['iphone']:
        sent.add(token.sent)
        #print(token.sent)
#         print("{0}/{1}/{2} <--{3}-- {4}/{5}/{6}".format(
#             token.text,token.pos_,token.tag_,token.dep_,token.head.text,token.head.pos_,token.head.tag_))
for s in sent:
    print(s)

The iPhone 11 Pro looks almost exactly like the iPhone XS from the front.
iPhone 11 Pro Max is a bad name.
Of course, Apple and Google are still in a fight over video codecs, so you can't watch any 4K or HDR content on the iPhone 11 Pro from YouTube.
I never really bought the idea that previous iPhone displays were Dolby Vision, but the iPhone 11 Pro display kind of pulls it off.
The standard iPhone 11 offers almost everything you get from the Pro for $699.
The iPhone 11 is the reverse.
Inside the new iPhone 11 Pro, there's the new A13 Bionic processor which, as usual, benchmarks far above the competition.
The XS Max has pulled up shadows so much that it looks a little hazy, while the iPhone 11 evenly pulls up shadows in Mariya's face, but preserves the contrast with the background.
If it improves photos taken in this kind of light, then the iPhone will basically pull ahead in every situation.
Instead, it's getting that number by taking its huge data set of iPhone usage and applying it

In [48]:
sent = set()
text = "it's Google pixel for day and of course as you should know we've got every single version sitting in front of me this every single version right well okay so it's the for excel in the three different colors they always got fancy names for the colors a clearly white oh so orange just black and so we have the Excel versions on the top and then below that you wouldn't know from the size of the box but we have the standard size pixel for some specifications for each here the XL 6.3 inch display 128 gigs of storage on this particular model the non XL version pixel for 5.7 inch display this is fresh it's hot off the presses so this video it's just it's an unboxing video and a first-look type of video just keep that in mind f HD Plus flexible OLED 444 PPI and then qhd+ so a higher pixel density on the XL at 5:37 DPI Gorilla Glass 5 on both oh they're gonna have 90 Hertz smooth displays you know that kind of quickie buttery what about a milkshake yeah is that smooth not bad the little guy is 2,800 milliamp hours at the big one 3700 both 18 watt fast charging and wireless charging and then of course a big thing near the cameras the brand new design on the back now they're like no we're gonna give you the versatility of multiple camera lenses white pixel for excel and the first thing you're gonna notice here a similar effect to the Panda version white on the back but then you get the black cutout for the cameras you get the black Ridge on the frame this is all metal and then the entire front of the device is black so it's a panda effect black and white I think a lot of people I think some people gonna like that mmm-hmm a USB type-c cable as well as a USB adapter and then this is the aforementioned power brick with the quick charge capability that's also orange this is for the for those that want to make more of a statement that's obviously a look I should also mention at this point it's a glass back it's a matte finish it's not a glossy finish oh yeah you're right it is glossy so on the black version the power switch is going to be this white color and on the other two versions actually and then on the orange phone it gets a little bit lighter in color that's yeah interesting choices here this is a very simple look to it but of course with any glossy finish you're opening yourself up to finger prints and so forth so you guys decide it looks fancy though this form factor is gonna be familiar to you it's a very practical utility more than anything easier to hold yes will he do ladies and gentlemen this is the small one I'll just place that there's your comparison right there big and small I'm gravitating towards this color there we go put that guy right on top I look down okay [Music] I notice straightaway here that the emphasis unlike a lot of other smartphone makers is not on screen to body ratio they're they're cool with having a forehead a chin on the device not overwhelmingly slim bezels a lot of that has to do with packing extra tech into the top forehead section of the device to NIR cameras dot projector flood emitter a camera as well with a 90 degree field of view fixed focus the pixel 4 &amp; 4 XL are now exclusively face on lock no fingerprint on the back no fingerprint in the display face unlock is enough and I guess they believe that's the future going forward with this degree of technology being baked in and giving up so much in the screen to body ratio department they've got to be cool with it and think that's the way to go so let's go ahead and give it a give it a test I'm going to scan my face in here he's locked phone unlocked phone locked phone unlocked phone I mean you just have to commit more space to have the extra tech built-in but you get you get stuff like the ability to illuminate the face in darker environments which if face unlock is going to be the only unlock available exclusively you kind of need that we can load up the front-facing camera real quick now you get the extra camera on the back and the singular camera on the front so it's a it's a switch obviously you have portrait you have night sight built-in all right this is a fresh device it's possibly not even final software number two all right so within the options of course you can configure the amount of processing that takes place this is with some of the natural face stuff turned off we have an abundance of boxes here so let's give that a quick shot as you can see here you get the wider look or you can move in a little bit closer let's try a 4x zoom I mean it's interesting because in the preview you can actually see some of the noise and stuff that you would traditionally have with a digitally zoomed image but then in the finished product the software runs such a process on it it eliminates all of that you can see a couple of cutouts on the bottom presumably for your speaker and microphone as well as your type-c connector interestingly in the spec sheet it lists a nano-sim or an e sim configurability alright so since we have the two displays stacked up I think that's gonna be an interesting characteristic for a lot of people mostly because they've added 90 Hertz to it now so what that means for people that are unfamiliar is you get this really twitchy kind of input really responsive field in fact it makes the entire phone feel faster alright maybe we should play a video let's just compare the speakers on the two units here I've got one of these surface laptop threes coming in well you didn't know that Jack caught you off-guard you know that a significant amount of sound is coming out of two different ports substantial sound from each area it's more volume than you might be used to out of the both of them balanced let's compare it to the little phone it all sounds very good on paper of course the marketplace will wait but talking a lot recently about trying to nail down the perfect laptop it's a real mission of mine so the larger than usual forehead there's also sensors in there capable of motion tracking tracking of a hand gestures Google's calling it motion sense because this software is early I can't showcase it to you at the moment but it's another trade-off I mean you're packing in some extra hardware for the potential to interact with your device without touching it beyond just assistant in your voice now potentially I'm scrolling and swiping without even touching it on the final version of the software it's also important to know I mean with the pixel devices in general it means you're getting the latest software that means you're getting features first including the latest when it comes to assistant ok Google what is unbox therapy and you can see the way that its installed the way that it's implemented is different than it used to be a lot happens now right within the main screen so you don't jump over to a different app in order to engage it and of course the advanced version of that is the continuation of conversation which is the initial instruction and then the deeper dive with assistant understanding context so you want to say okay Google hey Siri merely is often I think the biggest takeaway here is going to be Google moving towards 90 Hertz display for one and then secondary it's going to be camera performance eventually on final software what it's capable of from a computational perspective that's every single one on box as I mentioned this is just a first-look type of video I'll have a follow-up today's pixel day it just came out so you have two clearly white excel oh I should also mention that oh so orange is apparently limited edition so if you're interested I don't know what you guys do line up get your order in go black version which is the only glossy one of the bunch if that's what you're into polished fancy type of lifestyle you can go for that look as well I think I'm leaning towards this one right"
doc = nlp(text)
for token in doc:
    if token.text.lower() in ['pixel']:
        sent.add(token.sent)
for s in sent:
    print(s)

the XL 6.3 inch display 128 gigs of storage on this particular model the non XL version pixel for 5.7 inch display
lenses white pixel for excel and the first thing you're gonna notice here a similar effect to the Panda version white on the back
f HD Plus flexible OLED 444 PPI and then qhd+ so a higher pixel density on the XL at 5:37 DPI Gorilla Glass 5 on both
a 90 degree field of view fixed focus the pixel 4 &amp; 4 XL are now exclusively face on lock no fingerprint on the back no fingerprint in the display
but we have the standard size pixel for some specifications for each here
Google hey Siri merely is often I think the biggest takeaway here is going to be Google moving towards 90 Hertz display for one and then secondary it's going to be camera performance eventually on final software what it's capable of from a computational perspective that's every single one on box as I mentioned this is just a first-look type of video I'll have a follow-up today's pixel day
I mean with the pixe

In [57]:
list_sentences = list(sent)
for sen in list_sentences:
    print(sen)

The iPhone 11 Pro looks almost exactly like the iPhone XS from the front.
iPhone 11 Pro Max is a bad name.
Of course, Apple and Google are still in a fight over video codecs, so you can't watch any 4K or HDR content on the iPhone 11 Pro from YouTube.
I never really bought the idea that previous iPhone displays were Dolby Vision, but the iPhone 11 Pro display kind of pulls it off.
The standard iPhone 11 offers almost everything you get from the Pro for $699.
The iPhone 11 is the reverse.
Inside the new iPhone 11 Pro, there's the new A13 Bionic processor which, as usual, benchmarks far above the competition.
The XS Max has pulled up shadows so much that it looks a little hazy, while the iPhone 11 evenly pulls up shadows in Mariya's face, but preserves the contrast with the background.
If it improves photos taken in this kind of light, then the iPhone will basically pull ahead in every situation.
Instead, it's getting that number by taking its huge data set of iPhone usage and applying it

In [63]:
import statement_extractor
se = statement_extractor.StatementExtracter()
statements = se.is_statements('In fact, the cameras on the iPhone 11 and 11 Pro are so improved that I think theyre worth the year-over-year upgrade from last years models for the first time in a long time.')

# list_sentences = list(sent)
# for sen in list_sentences:
#     statements = se.is_statements(sen)
#     print(statements)

[In, fact, ,, the, cameras, on, the, iPhone, 11, and, 11, Pro, are, so, improved, that, I, think, they, re, worth, the, year, -, over, -, year, upgrade, from, last, years, models, for, the, first, time, in, a, long, time, .]


In [ ]:
1. Add punctuations
2. find sentences with the product 
3. run the is/are/has extractor get the sentences and run the bilstm/pointer generator for summary on sentences
4. 